In [132]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

reload(sys)
sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

from nltk.tokenize import *

In [133]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY
poets = db.POET
poems = db.POEM
rawpoet = pd.DataFrame(list(poets.find()))
rawpoem = pd.DataFrame(list(poems.find()))

### clean rawpoet table. At the moment, _id, poet_positions, & posyears (position years) column values appear unusable
### drop these columns for now

In [134]:
ctodrop = ['_id', 'poet_positions', 'poet_posyears']
rawpoet.drop(ctodrop, 1,inplace = True)

In [135]:
ctostr = ['movement_name','poet_bio','poet_name', 'poet_basicbio', 'poet_tags']

for i in ctostr:
    rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))

In [136]:
# rawpoet.head()[:1]

,movement_name,movement_url,poet_basicbio,poet_bio,poet_dob2,poet_name,poet_poems_url,poet_tags,poet_url
0,Slam/Spoken Word,http://www.poets.org/poetsorg/poets?field_scho...,,Taylor Mali is a four-time National Poetry Sla...,NaN,Taylor Mali,http://www.poets.org/poetsorg/poems/406436,\n \n ...,http://www.poets.org/poetsorg/poet/taylor-mali


###the poet's movement name tag is very important, we need to clean (remove dup tags), separate out into columns, then merge back into a clean poet table

In [138]:
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(str(x))),axis=1)
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(x)),axis=1)
rawpoet['tags'] = rawpoet['tags'].apply(lambda x: x.replace('-','').replace(' ','').split("\n"))
rawpoet['tags']=rawpoet['tags'].map(lambda x: ''.join(x).split(","))

In [139]:
# rawpoet.tags[:10]

0    [Slam/SpokenWord]
1       [Contemporary]
2       [Objectivists]
3         [Surrealism]
4         [Surrealism]
5         [Surrealism]
6         [Surrealism]
7       [Objectivists]
8       [Objectivists]
9       [Objectivists]
Name: tags, dtype: object

In [140]:
ptags = rawpoet['tags'].apply(pd.Series)

In [141]:
ptags.columns = ['primary_tag', 'tag2', 'tag3','tag4']
ptags.head()

,primary_tag,tag2,tag3,tag4
0,Slam/SpokenWord,NaN,NaN,NaN
1,Contemporary,NaN,NaN,NaN
2,Objectivists,NaN,NaN,NaN
3,Surrealism,NaN,NaN,NaN
4,Surrealism,NaN,NaN,NaN


##now we need to look at poet births/deaths

In [142]:
def extract_birth(text, born = True):
    extracted = 0
    if str(text) != 'nan':
        if not born:
            if len(text)>1:
                extracted = int(text[1])
        else:           
            extracted = int(text[0])
    return extracted

In [143]:
rawpoet['yr_born'] = rawpoet['poet_dob2'].apply(lambda x: extract_birth(x))
rawpoet['yr_died'] = rawpoet['poet_dob2'].apply(lambda x: extract_birth(x,False))

In [144]:
rawpoet['basic_bio']=rawpoet['poet_basicbio'].apply(lambda x: x.replace('-',''))

poet_basicbio contains a secondary source of birth and death years. It also contains where the poet was born; city, state and country. We need to extract this information.

In [145]:
rawpoet['basic_bio']=rawpoet['poet_basicbio'].apply(lambda x: x.replace('-',''))

#extract anything that is a number
rawpoet['birthdeath'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'\d+', x))
#extract non-numeric characters
rawpoet['birthplace'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'[^\d]+', x))
rawpoet['birthplace'] = rawpoet['birthplace'].apply(lambda x: u''.join(x))

In [146]:
def extract_bio(text, part = 'city'):
    x = text.split(',')
    y = x[0].strip().split(' ')
    city = []
    state = ''
    country = ''
    for i in y:
        if len(i) == 2:
            state = i
        if len(i) > 2:
            city.append(i)
    if len(x)>1:
        country = x[1]
    
    if part == 'city':
        return ' '.join(city).strip()
    elif part == 'state':
        return state.strip()
    else:
        return country.strip()

In [147]:
rawpoet['city'] = rawpoet['birthplace'].map(lambda x: extract_bio(x))
rawpoet['state'] = rawpoet['birthplace'].map(lambda x: extract_bio(x, 'state'))
rawpoet['country'] = rawpoet['birthplace'].map(lambda x: extract_bio(x, 'country'))

In [150]:
rawpoet.columns
coltokeep= ['poet_name','poet_url','poet_poems_url','tags','poet_bio','yr_born','yr_died','city','state','country']
poetdf = pd.concat([rawpoet[coltokeep],ptags],axis=1)

In [151]:
poetdf['dead'] = np.where(poetdf['yr_died'] > 0,1,0)
poetdf['current_age'] = np.where(poetdf['dead'] == 0, (2015- poetdf['yr_born']), 0)
poetdf['age_dead'] = np.where(poetdf['dead'] == 1, (poetdf['yr_died']-poetdf['yr_born']), 0)
poetdf['current_age'] = np.where(poetdf['current_age'] == 2015, 0,poetdf['current_age'])

In [159]:
# rawpoem.head()
poemdf = rawpoem[['poet_poems_url','poem_title', 'poem_yrpub', 'poem_text', 'poem_link']]
resultdf = pd.merge(poetdf, poemdf, how='inner', on=['poet_poems_url'])

In [162]:
resultdf.to_pickle('resultdf.pkl')

Variables:

* year published
* total number of words
* avg words per new line
* total number of stanzas
* avg newlines per stanza
* if first word of each line starts with capital letter
* number of words with capital letters
* number of commas
* commas per line
* number of periods
* number of exclamations
* number of pronoun references
* if i, my is used bool
* type token ratio
* concrete words/words overall
* abstract/words overall
* number of named entities/total words
* alliteration
* number of sensory words
* topic modeling - political

In [161]:
resultdf.columns
# rawpoem.columns


Index([     u'poet_name',       u'poet_url', u'poet_poems_url',
                 u'tags',       u'poet_bio',        u'yr_born',
              u'yr_died',           u'city',          u'state',
              u'country',    u'primary_tag',           u'tag2',
                 u'tag3',           u'tag4',           u'dead',
          u'current_age',       u'age_dead',     u'poem_title',
           u'poem_yrpub',      u'poem_text',      u'poem_link'],
      dtype='object')